### Match and questionnaire data from speed dating experiment run by Columbia professors Ray Fisman and Sheena Iyengar

https://data.world/annavmontoya/speed-dating-experiment

What influences love at first sight? (Or, at least, love in the first four minutes?) This dataset was compiled by Columbia Business School professors Ray Fisman and Sheena Iyengar for their paper Gender Differences in Mate Selection: Evidence From a Speed Dating Experiment.

Data was gathered from participants in experimental speed dating events from 2002-2004. During the events, the attendees would have a four minute "first date" with every other participant of the opposite sex. At the end of their four minutes, participants were asked if they would like to see their date again. They were also asked to rate their date on six attributes: Attractiveness, Sincerity, Intelligence, Fun, Ambition, and Shared Interests.

The dataset also includes questionnaire data gathered from participants at different points in the process. These fields include: demographics, dating habits, self-perception across key attributes, beliefs on what others find valuable in a mate, and lifestyle information. See the Speed Dating Data Key document below for details.

For more analysis from Iyengar and Fisman, read Racial Preferences in Dating.

In [63]:
dfo = pd.read_csv('Speed Dating Data.csv', encoding='latin-1')
dfo.shape

(8378, 195)

In [75]:
import json
dk = json.load(open('key.json'))
dk = {k: {int(kk): vv for kk, vv in v.items()} for k, v in dk.items()}

In [76]:
c = list(dk.keys())

In [81]:
for k, v in dk.items():
    if len(v) > 0:
        print('mapping {}'.format(k))
        df[k] = dfo[k].map(lambda x: v[x] if not np.isnan(x) else x)
df = df[c]

mapping condtn
mapping race
mapping field_cd
mapping go_out
mapping gender
mapping career_c
mapping date
mapping goal
mapping samerace


In [91]:
for k in ['idg', 'iid', 'round', 'partner', 'position']:
    df[k] = df[k].astype('category')
for k in ['order']:
    df[k] = df[k].astype('float')

In [92]:
for k in df:
    if df[k].dtype.name == 'object':
        df[k] = df[k].astype('category')
    if df[k].dtype.name.startswith('int'):
        print(k, df[k].min(), df[k].max())

match 0 1
dec_o 0 1


In [84]:
df.head(n=1).T

,0
dining,9
idg,1
field,Law
pf_o_att,35
condtn,limited choice
movies,10
exphappy,3
mn_sat,NaN
gaming,1
iid,1


In [54]:
df.columns.tolist()

['iid',
 'id',
 'gender',
 'idg',
 'condtn',
 'wave',
 'round',
 'position',
 'positin1',
 'order',
 'partner',
 'pid',
 'match',
 'int_corr',
 'samerace',
 'age_o',
 'race_o',
 'pf_o_att',
 'pf_o_sin',
 'pf_o_int',
 'pf_o_fun',
 'pf_o_amb',
 'pf_o_sha',
 'dec_o',
 'attr_o',
 'sinc_o',
 'intel_o',
 'fun_o',
 'amb_o',
 'shar_o',
 'like_o',
 'prob_o',
 'met_o',
 'age',
 'field',
 'field_cd',
 'undergra',
 'mn_sat',
 'tuition',
 'race',
 'imprace',
 'imprelig',
 'from',
 'zipcode',
 'income',
 'goal',
 'date',
 'go_out',
 'career',
 'career_c',
 'sports',
 'tvsports',
 'exercise',
 'dining',
 'museums',
 'art',
 'hiking',
 'gaming',
 'clubbing',
 'reading',
 'tv',
 'theater',
 'movies',
 'concerts',
 'music',
 'shopping',
 'yoga',
 'exphappy',
 'expnum',
 'attr1_1',
 'sinc1_1',
 'intel1_1',
 'fun1_1',
 'amb1_1',
 'shar1_1',
 'attr4_1',
 'sinc4_1',
 'intel4_1',
 'fun4_1',
 'amb4_1',
 'shar4_1',
 'attr2_1',
 'sinc2_1',
 'intel2_1',
 'fun2_1',
 'amb2_1',
 'shar2_1',
 'attr3_1',
 'sinc3_1',
 

In [41]:
df[list(dk.keys())]

,gender,race,date,field_cd,career_c,condtn,samerace,go_out,goal
0,0,4.0,7.0,1.0,NaN,1,0,1.0,2.0
1,0,4.0,7.0,1.0,NaN,1,0,1.0,2.0
2,0,4.0,7.0,1.0,NaN,1,1,1.0,2.0
3,0,4.0,7.0,1.0,NaN,1,0,1.0,2.0
4,0,4.0,7.0,1.0,NaN,1,0,1.0,2.0
5,0,4.0,7.0,1.0,NaN,1,0,1.0,2.0
6,0,4.0,7.0,1.0,NaN,1,0,1.0,2.0
7,0,4.0,7.0,1.0,NaN,1,0,1.0,2.0
8,0,4.0,7.0,1.0,NaN,1,0,1.0,2.0
9,0,4.0,7.0,1.0,NaN,1,0,1.0,2.0


In [36]:
df.go_out.dtype.name

'object'

In [18]:
df.dtypes

iid           int64
id          float64
gender        int64
idg           int64
condtn        int64
wave          int64
round         int64
position      int64
positin1    float64
order         int64
partner       int64
pid         float64
match         int64
int_corr    float64
samerace      int64
age_o       float64
race_o      float64
pf_o_att    float64
pf_o_sin    float64
pf_o_int    float64
pf_o_fun    float64
pf_o_amb    float64
pf_o_sha    float64
dec_o         int64
attr_o      float64
sinc_o      float64
intel_o     float64
fun_o       float64
amb_o       float64
shar_o      float64
             ...   
amb1_3      float64
shar1_3     float64
attr7_3     float64
sinc7_3     float64
intel7_3    float64
fun7_3      float64
amb7_3      float64
shar7_3     float64
attr4_3     float64
sinc4_3     float64
intel4_3    float64
fun4_3      float64
amb4_3      float64
shar4_3     float64
attr2_3     float64
sinc2_3     float64
intel2_3    float64
fun2_3      float64
amb2_3      float64
